# 1. Importación de librerías

**Nota**: En caso de necesitar instalar las librerías se puede ejecutar el siguiente bloque antes de realizar las importaciones:

In [ ]:
!pip install opencv-python==4.10.0.82 numpy==1.23.5 mediapipe==0.10.14 tensorflow==2.12.0rc0

In [1]:
import os
import cv2
import time
import math
import numpy as np
import mediapipe as mp
import tensorflow as tf
from datetime import datetime
from matplotlib import pyplot as plt

# Para la partición de datos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Para la creación del modelo con LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

# Para la evaluación del modelo con LSTM
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

# 2. Uso de OpenCV para detectar partes del cuerpo

## 2.1. Detección de puntos de interés (landmarks) de cara, brazos y manos con OpenCV 

In [2]:
def body_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Se convierte porque CV2 trabaja con BGR y el modelo con RGB
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Se regresa al valor por defecto BGR de CV2

    return image, results

In [3]:
def draw_landmarks(image, results):
    mp_draw.draw_landmarks(
        image, 
        results.face_landmarks, 
        mp_hol.FACEMESH_TESSELATION,
        mp_draw.DrawingSpec(color=(255,255,255), thickness=1, circle_radius=1),
        mp_draw.DrawingSpec(color=(255,255,255), thickness=1, circle_radius=1)
    )
    mp_draw.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_hol.POSE_CONNECTIONS,
        mp_draw.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=1),
        mp_draw.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=1)
    )
    mp_draw.draw_landmarks(
        image, 
        results.left_hand_landmarks, 
        mp_hol.HAND_CONNECTIONS,
        mp_draw.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1),
        mp_draw.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1)
    )
    mp_draw.draw_landmarks(
        image, 
        results.right_hand_landmarks, 
        mp_hol.HAND_CONNECTIONS,
        mp_draw.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1),
        mp_draw.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1)
    )

## 2.2. Extracción de puntos de interés (landmarks) a un arreglo de numpy para ser utilizado por la LSTM neural network

In [4]:
def get_landmarks_array(results):
    face = np.array([[r.x, r.y, r.z] for r in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3) 
    pose = np.array([[r.x, r.y, r.z, r.visibility] for r in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4) 
    left_hand = np.array([[r.x, r.y, r.z] for r in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[r.x, r.y, r.z] for r in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([face, pose, left_hand, right_hand])

In [5]:
mp_hol = mp.solutions.holistic
mp_draw = mp.solutions.drawing_utils
holistic = mp_hol.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7)

**Nota**: Este bloque de código es para verificar que los landmarks se están mostrando correctamente en la pantalla, se puede omitir en la ejecución del programa

In [8]:
cap = cv2.VideoCapture(0)

width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height

while cap.isOpened():
    # Obtiene la imagen de la camara
    ret, frame = cap.read()

    # Realiza la detección de rostro, brazos y manos con mediapipe
    image, results = body_detection(frame, holistic)
    
    # Dibuja los puntos detectados con el modelo holistico y los agrega a la captura de video
    draw_landmarks(image, results)

    cv2.putText(image, f'Size {width}x{height}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
    
    # Muestra captura de video en pantalla
    cv2.imshow('OpenCV', image)

    # En caso de oprimir ESC, sale del While
    if cv2.waitKey(27) & 0xFF == ord('\x1b'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\abner\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


# 3. Generación de conjunto de datos para el entrenamiento y validación

**Nota**: las secciones 3.2. y 3.3. pueden omitirse si ya se generaron datos de entrenamiento

## 3.1. Configuración de la carpeta de datos

In [6]:
# Variables de open CV y MediaPipe para obtener los landmarks
cap = cv2.VideoCapture(0)
mp_hol = mp.solutions.holistic
mp_draw = mp.solutions.drawing_utils
holistic = mp_hol.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Ruta de la carpeta de datos
folder_path = os.path.join('data')

# Configuracion del numero de secuencias a generar por cada gesto y la cantidad de frames a recolectar por cada secuencia
gestures = np.array(os.listdir(folder_path))
seq_amount = 30 # 30 secuencias por gesto
seq_length = 30 # 30 frames por secuencia

## 3.2. Creación de carpetas

In [ ]:
for g in gestures:
    for s in range(seq_amount):
        try:
            os.makedirs(os.path.join(folder_path, g, str(s)))
        except:
            pass

## 3.3. Programa para la captación de datos desde la cámara web

In [35]:

for g in gestures:
    for s in range(seq_amount):
        for frame_number in range(seq_length):
            
            # Obtiene la imagen de la camara
            ret, frame = cap.read()
        
            # Realiza la detección de rostro, brazos y manos con mediapipe
            image, results = body_detection(frame, holistic)
            
            # Dibuja los puntos detectados con el modelo holistico y los agrega a la captura de video
            draw_landmarks(image, results)

            # Inicia la recolección de datos de los gestos, muestra en pantalla mensaje para hacer gestos y espera 1 segundo para tomar nuevos datos
            if frame_number == 0:
                cv2.putText(image, 'Starting data gathering', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 4, cv2.LINE_AA)
                cv2.putText(image, f'Gathering frames for {g} - Video No. {s}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
                cv2.waitKey(1000)
            else:
                cv2.putText(image, f'Gathering frames for {g} - Video No. {s}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)

            # Guarda los datos obtenidos en un archivo
            keypoints = get_landmarks_array(results)
            output_path = os.path.join(folder_path, g, str(s), str(frame_number))
            np.save(output_path, keypoints)
            
            # Muestra captura de video en pantalla
            cv2.imshow('OpenCV', image)
        
            # En caso de oprimir ESC, sale del While
            if cv2.waitKey(27) & 0xFF == ord('\x1b'):
                break

cap.release()
cv2.destroyAllWindows()

## 3.4. Programa para la captación de datos desde la carpeta de videos

In [ ]:
videos_path = os.path.join('videos')
output_videos_path = os.path.join('data_videos')

for gesture in os.listdir(videos_path):
    for video in os.listdir(os.path.join(videos_path, gesture)):
        cap = cv2.VideoCapture(os.path.join(videos_path, gesture, video))
        
        print(f'Video: {gesture}\nTotal de frames: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}\nTomar un frame cada {int(cap.get(cv2.CAP_PROP_FRAME_COUNT))/seq_length} frames')

        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                print("Can't receive frame (stream end?). Exiting ...")
                break
            
            # Cambia la resolucion a 640x480 px
            current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            frame = cv2.resize(frame, (640,480))

            # Realiza la detección de rostro, brazos y manos con mediapipe
            image, results = body_detection(frame, holistic)

            # Dibuja los puntos detectados con el modelo holistico y los agrega a la captura de video
            draw_landmarks(image, results)

            cv2.putText(image, f'Frame {current_frame} of {cap.get(cv2.CAP_PROP_FRAME_COUNT)}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)                    
            
            # Como cada secuencia tiene 30 frames, se recortan algunos frames de forma dinámica
            if (current_frame-1) % math.floor((cap.get(cv2.CAP_PROP_FRAME_COUNT)/seq_length)) == 0:
                # Guarda los keypoints en archivos
                keypoints_videos = get_landmarks_array(results)
                output_path = os.path.join(output_videos_path, gesture, video.split('.')[0])
                if not os.path.exists(output_path):
                    try:
                        os.makedirs(output_path)
                    except:
                        pass                    
                np.save(os.path.join(output_path, str(current_frame-1)), keypoints_videos)

                cv2.imshow('frame', image)

            if cv2.waitKey(1) == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

## 3.5. Eliminación de frames sobrantes por cada video, a modo de tener un número fijo de frames

In [ ]:
def get_id(filename):
    return int(filename.split('.')[0])

# Recorre cada una de las carpetas para eliminar los primeros N y últimos M frames guardados, a modo de sólo dejar 30 frames por video
for gesture in os.listdir(output_videos_path):
    for video in os.listdir(os.path.join(output_videos_path, gesture)):
        files = sorted(os.listdir(os.path.join(output_videos_path, gesture, video)), key=get_id)
        n_files_detele = len(files)-seq_length
        
        if n_files_detele > 0:
            print('Deleting files...')
            if n_files_detele % 2 == 0:
                print(f'Deleting {n_files_detele} - {int(n_files_detele/2)} at the beginig and at the end')
                files_to_delete = files[:int(n_files_detele/2)] + files[-int(n_files_detele/2):]
                print('Deleting files', files_to_delete)
                for f in files_to_delete:
                    os.remove(os.path.join(output_videos_path, gesture, video, f))
            else:
                print(f'Deleting {n_files_detele} - {math.floor(n_files_detele/2)} at the beginig and {math.ceil(n_files_detele/2)} at the end')
                files_to_delete = files[:math.floor(n_files_detele/2)] + files[-math.ceil(n_files_detele/2):]
                print('Deleting files', files_to_delete)
                for f in files_to_delete:
                    os.remove(os.path.join(output_videos_path, gesture, video, f))
        else:
            print('Already 30 files per sequence')

        print('Renaming files...')
        files = sorted(os.listdir(os.path.join(output_videos_path, gesture, video)), key=get_id)
        name_seq = 0
        for index, file in enumerate(files):
            print(f'Renaming {file} to {index}.npy')
            os.rename(os.path.join(output_videos_path, gesture, video, file), os.path.join(output_videos_path, gesture, video, f'{index}.npy'))


# 4. Preprocesamiento y preparación de datos

## 4.1. Obtención de los datos generados en la sección 3. Generación de conjunto de datos para el entrenamiento y validación

In [13]:
label_map = {label:num for num, label in enumerate(gestures)}
sequences = []
labels = []

for gesture in gestures:
    for sequence in range(seq_amount):
        window = []
        for frame_number in range(seq_length):
            res = np.load(os.path.join(folder_path, gesture, str(sequence), f"{frame_number}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[gesture])

## 4.2. Partición del conjunto de datos en datos de entrenamiento y datos de prueba

In [14]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

# 5. Construcción del modelo con LSTM para la predicción de gestos en lengua de signos

## 5.1. Definición de los parámetros para guardar los logs en TensorBoard

In [9]:
logs_path = os.path.join('logs')
tb_callback = TensorBoard(log_dir=logs_path)

## 5.2. Configuración del modelo con LSTM

In [10]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(gestures.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

## 5.3. Entrenamiento del modelo de predicción de gestos en lengua de signos

**Nota**: En caso de tener un modelo ya entrenado, se puede omitir esta parte y pasar a la sección 5.4. 

In [19]:
model.fit(x_train, y_train, epochs=500, callbacks=[tb_callback])
filename = 'predictor_LSE_'+datetime.now().strftime('%d-%m-%Y_%H%M')+'.h5'
model.save(filename)

Epoch 1/500
6/6 [==============================] - 0s 67ms/step - loss: 0.4001 - categorical_accuracy: 0.8148
Epoch 2/500
6/6 [==============================] - 0s 76ms/step - loss: 0.4907 - categorical_accuracy: 0.7937
Epoch 3/500
6/6 [==============================] - 0s 71ms/step - loss: 0.4204 - categorical_accuracy: 0.8466
Epoch 4/500
6/6 [==============================] - 0s 67ms/step - loss: 0.3621 - categorical_accuracy: 0.8413
Epoch 5/500
6/6 [==============================] - 0s 71ms/step - loss: 0.2893 - categorical_accuracy: 0.8889
Epoch 6/500
6/6 [==============================] - 0s 67ms/step - loss: 0.2916 - categorical_accuracy: 0.8783
Epoch 7/500
6/6 [==============================] - 0s 68ms/step - loss: 0.2791 - categorical_accuracy: 0.8942
Epoch 8/500
6/6 [==============================] - 0s 68ms/step - loss: 0.2463 - categorical_accuracy: 0.9312
Epoch 9/500
6/6 [==============================] - 0s 69ms/step - loss: 0.2479 - categorical_accuracy: 0.9048
Epoch 10/5

## 5.4. Carga del modelo entrenado

In [15]:
model.load_weights('./predictor_LSE_28-06-2024_0456.h5')

## 5.5. Evaluación del modelo entrenado

In [16]:
pred_test = model.predict(x_test)
pred_test_real = np.argmax(y_test, axis=1).tolist()
pred_test = np.argmax(pred_test, axis=1).tolist()
print("Matriz de confusión: \n", multilabel_confusion_matrix(pred_test_real, pred_test))
print("\n Precisión: ", accuracy_score(pred_test_real, pred_test))

1/1 [==============================] - 0s 50ms/step
Matriz de confusión: 
 [[[19  0]
  [ 0  2]]

 [[16  0]
  [ 0  5]]

 [[19  0]
  [ 0  2]]

 [[19  0]
  [ 0  2]]

 [[18  0]
  [ 0  3]]

 [[15  0]
  [ 0  6]]

 [[20  0]
  [ 0  1]]]

 Precisión:  1.0


# 6. Pruebas de reconocimiento en tiempo real

In [17]:
def render_prediction(res, gestures, frame):
    colors = [(16,37,64),
              (21,49,84),
              (26,61,105),
              (31,73,125),
              (36,85,145),
              (41,97,166),
              (46,109,186)]
    output = frame.copy()
    for n, p in enumerate(res):
        cv2.rectangle(output, (0,60+n*40), (int(p*100), 90+n*40), colors[n], -1)
        cv2.putText(output, gestures[n], (0,85+n*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    return output

In [19]:
sequence = []
sentence = []
predictions = []
threshold = 0.999999
cap = cv2.VideoCapture(0)

while cap.isOpened():
    # Obtiene la imagen de la camara
    ret, frame = cap.read()

    # Realiza la detección de rostro, brazos y manos con mediapipe
    image, results = body_detection(frame, holistic)
    
    # Dibuja los puntos detectados con el modelo holistico y los agrega a la captura de video
    draw_landmarks(image, results)

    # Procesamiento de predicción
    landmarks = get_landmarks_array(results)
    sequence.append(landmarks)
    sequence = sequence[-30:]
    pred = model.predict(np.expand_dims(sequence, axis=0))[0]
    predictions.append(np.argmax(pred))

    if len(sequence) == 30:
        pred = model.predict(np.expand_dims(sequence, axis=0))[0]
        print(f'Prediction = {gestures[np.argmax(pred)]}, {pred}')
        predictions.append(np.argmax(pred))

    # Visualización de resultados
    if np.unique(predictions[-10:])[0]== np.argmax(pred):
        if pred[np.argmax(pred)] > threshold:
            if len(sentence) > 0:
                if gestures[np.argmax(pred)] != sentence[-1]:
                    sentence.append(gestures[np.argmax(pred)])
            else:
                sentence.append(gestures[np.argmax(pred)])
    
    if len(sentence) > 5:
        sentence = sentence[-5:]

    output_text = ' '.join(sentence)
    image = render_prediction(pred, gestures, image)
    # cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
    # cv2.putText(image, output_text, (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
        
    # Muestra captura de video en pantalla
    cv2.imshow('OpenCV', image)

    # En caso de oprimir ESC, sale del While
    if cv2.waitKey(27) & 0xFF == ord('\x1b'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 24ms/step
Prediction = bienvenido, [9.4822615e-01 5.9613737e-04 4.0957624e-05 3.3886219e-05 2.0557945e-03
 5.9579862e-03 4.3089055e-02]
1/1 [==============================] - 0s 21ms/step
Prediction = saludo, [7.4411023e-15 3.3506386e-25 1.5877652e-27 3.2353277e-21 3.1909540e-12
 8.7721717e-05 9.9991226e-01]
1/1 [==============================] - 0s 21ms/step
Prediction = saludo, [1.2711536e-13 6.0506130e-23 2.4694950e-25 7.4282427e-19 3.4405801e-11
 1.2165401e-04 9.9987829e-01]
1/1 [==============================] - 0s 22ms/step
Prediction = saludo, [8.6983080e-11 1.7791030e-19 1.7399978e-21 3.2754984e-16 3.2162881e-09
 2.2835161e-04 9.9977165e-01]
1/1 [==============================] - 0s 20ms/step
Prediction = saludo, [1.3242722e-08 4.6015770e-17 9.5589426e-19 1.4925598e-14 8.1043865e-08
 4.1323822e-04 9.9958664e-01]
1/1 [==============================] - 0s 24ms/step
Prediction = saludo, [2.9103253e-07 2.5872128e-15 9.1821191e-17 2.3828818e